# Pynq-Z1 HDMI Sobel Filter Demo

* Signal Chain: HDMI In -> RGB-to-Grayscale -> Sobel Filter -> HDMI Out 
* HDMI In/Out Resolution: 1080x1920 with 24bit per pixel
* Frame Rate: 32 frames per second
* Repository: https://github.com/beja65536/pz1_sobelfilter
* Based-on: http://www.wiki.xilinx.com/SDSoC+designs+and+platforms

### Install package

In [1]:
!sudo pip install --upgrade '/home/testgit/pz1_sobelfilter'

Processing /home/testgit/pz1_sobelfilter
  Running setup.py install for pz1sofi ... - \ | / done


### Step 1: Load Bitstream & Shared Library

In [3]:
from pynq import Overlay
from pz1sofi.general_const import *
Overlay(BITFILE).download()

In [4]:
import cffi
sds = cffi.FFI()
#Accelerator functions
sds.cdef("void _p0_rgb_2_gray_0(uint8_t * input,"+
         "uint8_t * output);")
sds.cdef("void _p0_sobel_filter_0(uint8_t * input,"+
         "uint8_t * output);")
#XLNK functions
sds.cdef("void *cma_alloc(uint32_t len, uint32_t cacheable);")
sds.cdef("void cma_free(void *buf);")

libsds = sds.dlopen(LIBRARY)

### Step 2: Initialize HDMI I/O & Memory

In [ ]:
from pynq.drivers.video import HDMI
hdmi_out = HDMI('out')
hdmi_in = HDMI('in', init_timeout=10)
hdmi_in.start()
hdmi_out.start()

In [ ]:
frame_in = sds.cast("uint8_t *",hdmi_in.frame_addr())
frame_out = sds.cast("uint8_t *",hdmi_out.frame_addr())
frame_gray = sds.cast("uint8_t *",libsds.cma_alloc(1920*1080,0))

### Step 3: Run OpenCV Sobel Filter

In [ ]:
import numpy as np
import time
import cv2
num_frames = 16

In [ ]:
#Sobel Filter
time1 = time.time()
for i in range (num_frames):    
    np_frame = (np.frombuffer(hdmi_in.frame_raw(), 
                             dtype=np.uint8)).reshape(1080,1920,3)
    grad_x = cv2.Sobel(np_frame,cv2.CV_8U,1,0,ksize = 3, scale = 1, 
                       delta = 0,borderType = cv2.BORDER_DEFAULT)
    grad_y = cv2.Sobel(np_frame,cv2.CV_8U,0,1,ksize = 3, scale = 1, 
                       delta = 0, borderType = cv2.BORDER_DEFAULT)
    abs_grad_x = cv2.convertScaleAbs(grad_x)
    abs_grad_y = cv2.convertScaleAbs(grad_y)
    sobel_frame = cv2.addWeighted(abs_grad_x,0.5,abs_grad_y,0.5,0)
    hdmi_out.frame_raw(bytearray(sobel_frame))
time2 = time.time()
print("OpenCV: {0:.2f} fps".format(num_frames/(time2-time1)))

### Step 4: Run Hardware-accelerated Sobel Filter

In [ ]:
import time
num_frames = 320

In [ ]:
time1 = time.time()
for i in range(num_frames):
    libsds._p0_rgb_2_gray_0(frame_in,frame_gray)
    libsds._p0_sobel_filter_0(frame_gray,frame_out)
time2 = time.time()
print("SDSoC: {0:.2f} fps".format(num_frames/(time2-time1)))

### Step 5: Release Resources

In [ ]:
hdmi_out.stop()
hdmi_in.stop()
libsds.cma_free(frame_in)
libsds.cma_free(frame_out)
libsds.cma_free(frame_gray)
del hdmi_in
del hdmi_out